# NID (NSO in Docker) demo
## 🦴 NID skeletons 🦴
---
### We need consistency in our development/testing environments

Fresh new project. The team expects lots from you. It is imperative that you get going as fast as possible.

But wait, you need a development environment to get started, right?

The following questions may arise:
- Which is our current NSO version?
- Which NEDs are we using?
- Are there any standard, often inmutable support services that we're using? (utils, rfs, CFP)

Setting up these environments often takes plenty of time.<br>On top of that, the list of requirements is often blurry and prone to errors (a.k.a Not the most updated project documentation).

The environment issue gets more interesting when we think about testing.<br>The same questions arise when it comes to setting up our staging so that it is alligned to what is out there in production.<br>Confusions regarding the versions can lead to consequences when releasing service updates.

With this in mind, the NID project encompasses a series of utilities for creating the so-called "skeletons", which tackle the needing of consistent yet flexible staging environments in a programmatic way.

To get started, follow the steps for generating your ```base``` and ```dev``` images. Your image repository should include them by now.

In [1]:
!docker images | grep cisco-nso

cisco-nso-base                                                   6.0-root               a173e5c9baa9   3 months ago    678MB
cisco-nso-dev                                                    6.0-root               825b256d44ac   3 months ago    1.43GB


### 📦 ```package``` skeleton 📦
---

This option allows you to generate a NSO image based on the previously generated ones with a persistent and compiled service.

Navigate to the following location of the ```nso-docker``` repository and copy your rfs, CFP, etc project:

```
/nso-docker/skeletons/package/packages
```

In our example, we are using a very simple rfs called rfs-common.
Now, navigate to the parent ```package``` folder and execute the following command:

```
make build NSO_VERSION=6.0-root
```

This will render the following images:

In [26]:
!docker images | grep package/

package/package                                                  6.0-root-alfsando      b2baaf804c17   3 hours ago      5.33kB
package/package                                                  MM_6.0-root-alfsando   b2baaf804c17   3 hours ago      5.33kB
package/testnso                                                  6.0-root-alfsando      ddbca12cb585   3 hours ago      678MB
package/build                                                    6.0-root-alfsando      a0d3c6dfb8ed   3 hours ago      1.43GB


What just happened? The mentioned rfs was compiled, and a series of NSO docker images were created with this pre-loaded, already-compiled and onboarded service within.

Let's spin one of these and see the contents:

In [14]:
!docker run -itd --env-file pipeline-utils/nso_setup.list --platform=linux/amd64 --name MYNSO-RFS package/testnso:6.0-root-alfsando

9d8f20264639559da2bf0bf846183bf478fb8a7d19cbd41a7b26c01dc5b75810


Voilà! NSO with a persistent package was just spun up. Let's inspect the contents.

**⏳ IMPORTANT! ⏳**: Let your container cool for a little bit before attempting any NSO-related operation. The ```testnso``` flavour that we just spun will start NSO as soon as it is deployed, but still it takes a little while.

In [34]:
!docker exec -i MYNSO-RFS bash -l -c "echo 'show packages package * package-version' | ncs_cli -Cu admin"

            PACKAGE  
NAME        VERSION  
---------------------
rfs-common  1.0      



However, this rfs is not available in the packages folder! This is to respect the concept of a NID skeleton, as this rfs shall be inmutable for this NID Docker image. Any changes to the rfs should be done by generating a new NID image. Any other packages can be added to the ```/nso/run/packages/``` location.

In [20]:
!docker exec -i MYNSO-RFS bash -l -c "ls /nso/run/packages/"

### 🚙 ```ned``` skeleton 🚙
---

The same concept applies for adding NEDs into our NID images. Navigate to the following location of the ```nso-docker``` repository and copy your unzipped NED folder:

```
/nso-docker/skeletons/ned/packages
```

In our example, we are using the ```cisco-iosxr-cli-7.43``` NED.
Now, navigate to the parent ```ned``` folder and execute the following command:

```
make build NSO_VERSION=6.0-root
```

What will happen now is that the NED will be compiled and onboarded into our NSO image, so everytime that we spin a new container based on it, we don't need to onboard this NED.

Unfortunately, the compilation is unavoidable, hence this process takes some time ...

#### But wait! There's more

Not only will you get a NSO image with an onboarded NED, but also an image of the NED per se! You can now spin up individual NED devices that you can onboard individually into your NSO container!

Let's have a look at the rendered images:

In [36]:
!docker images | grep ned/

ned/package-cisco-iosxr-cli-7.43                                 6.0-root-alfsando      cecfdbcc8adf   21 minutes ago   53.2MB
ned/package-cisco-iosxr-cli-7.43                                 MM_6.0-root-alfsando   cecfdbcc8adf   21 minutes ago   53.2MB
ned/netsim-cisco-iosxr-cli-7.43                                  6.0-root-alfsando      69591c1be6f1   21 minutes ago   1.48GB
ned/netsim-cisco-iosxr-cli-7.43                                  MM_6.0-root-alfsando   69591c1be6f1   21 minutes ago   1.48GB
ned/netsim                                                       6.0-root-alfsando      69591c1be6f1   21 minutes ago   1.48GB
ned/netsim                                                       MM_6.0-root-alfsando   69591c1be6f1   21 minutes ago   1.48GB
ned/package                                                      6.0-root-alfsando      e3d429421f74   21 minutes ago   53.2MB
ned/package                                                      MM_6.0-root-alfsando   e3d429421f74   21 minut

Let's spin a container with the image which contains NSO with the onboarded NED:

In [39]:
!docker run -itd --env-file pipeline-utils/nso_setup.list --platform=linux/amd64 --name MYNSO-NED ned/testnso:6.0-root-alfsando

3a42b8add6da91d85f4c1c5a151072c6be0df831f7c4a5e8ddb45181e52b49e8


Let's have a look at the packages installed:

In [40]:
!docker exec -i MYNSO-NED bash -l -c "echo 'show packages package * package-version' | ncs_cli -Cu admin"

                      PACKAGE  
NAME                  VERSION  
-------------------------------
cisco-iosxr-cli-7.43  7.43     



Now, let's spin up a full NETSIM device based on the generted ```netsim/``` image:

In [4]:
!docker run -td --platform=linux/amd64 -p 222:22 --name IOSXR-0 ned/netsim-cisco-iosxr-cli-7.43:6.0-root-alfsando

b425787b06b5d2aef6814faf095a2e4a8211e5b3f351999e1fe142c8a6337669


After waiting a little bit for it to warm up, we have a full netsim device for us to use!

Let's have a look at its device configs xml for further onboarding on NSO:

In [7]:
!docker exec -i IOSXR-0 bash -l -c "cd /opt/ncs/current/ && ncs-netsim ncs-xml-init > iosxr-0.xml"
!docker exec -i IOSXR-0 bash -l -c "cd /opt/ncs/current/ && cat iosxr-0.xml"


<devices xmlns="http://tail-f.com/ns/ncs">
   <device>
     <name>b425787b06b5</name>
     <address>127.0.0.1</address>
     <port>22</port>
     <ssh>
       <host-key>
         <algorithm>ssh-ed25519</algorithm>
         <key-data>ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAICp3BzdAicVmmH9CjKDfAstEEi48y3+rwA3YBh4fXoQS root@buildkitsandbox</key-data>
       </host-key>
       <host-key>
         <algorithm>ssh-rsa</algorithm>
         <key-data>ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDsa73No6ya/JHsyAFiaVPbU/GiZnTSTUaOmKCX75KkFWxG5prV0UWKdDd1r33dI9ImJ/NTXsGujJeg6Ln8qQkO7V7R7u/nbBCv5JMAFS42ssGbcddP+z5T9CADIm8xSC30W1xvoLw2552JQQ3cK+DKMyWVxO81PeN1Hf0Lev4EGgnKn92FZDfPNv8NA8YT5j/pYC1hI3CpiWyrBF13cUSTsz5UwAY9KHPUphVobzjecjB+nA/DZXTyHxAE3oI7nzcCegrZQ2da7XyMp02l82H8zYIbCzSAVlCY8pGJR9S/V2AdTTXFgWleulro+Gwy+ABKCLQepu7MmiPJ0HkX0QJ5 root@buildkitsandbox</key-data>
       </host-key>
     </ssh>
     <ssh-algorithms>
       <public-key>ssh-ed25519</public-key>
       <public-key>ssh-rsa</public-key>
     </s

You get the picture, right?

We can create as many netsim devices as we need, and treat them individually as independent containers which can be spun up and destroyed at will.

This is ideal for dynamic, ondemand testing scenarios such as a Testing Stage in a CICD pipeline.

Later on we will review a way of onboarding our netsims into our staging NSO container.

### 📚 But wait, can I have both? 📚
---

**YES!**. It is possible to integrate several pre-bundled elements in our skeletons, so that we can achieve a single golden NSO Docker image with little to no effort. Let's load our extracted NED and our rfs package in the following location of the ```nso-docker``` repository:

```
/nso-docker/skeletons/system/packages
```

Again, navigate to the parent ```system``` folder and issue the same command:

```
make build NSO_VERSION=6.0-root
```

This time you will have the following images generated:

In [2]:
!docker images | grep system/

system/nso                                                       6.0-root-alfsando      0f29fa22bb93   42 seconds ago   732MB
system/nso                                                       MM_6.0-root-alfsando   0f29fa22bb93   42 seconds ago   732MB
system/build                                                     6.0-root-alfsando      4f29b0a27c3a   47 seconds ago   1.52GB


Let's spin a container and see its contents:

In [3]:
!docker run -itd --env-file pipeline-utils/nso_setup.list --platform=linux/amd64 --name MYNSO-FULL system/nso:6.0-root-alfsando

0f7d676c9c83868b219a6b11be5bf0f3b7d0468a143aa275e0ad05d48bc3da5f


In [8]:
!docker exec -i MYNSO-FULL bash -l -c "echo 'show packages package * package-version' | ncs_cli -Cu admin"

                      PACKAGE  
NAME                  VERSION  
-------------------------------
cisco-iosxr-cli-7.43  7.43     
rfs-common            1.0      



### 🧩 Putting together a container-based testing/development scenario 🧩

The objective now is to generate a series of netsim "devices" (containers) and onboard them into our staging NSO container.

Certainly there are many ways of doing so. The following steps describe one possible solution to this task.

For that purpose, we need to first create a ```docker network``` so that we enable the communication between our netsims and the NSO container:

In [9]:
!docker network create nsotestenvnetwork

08ca453e44c7d6aa29ae79d8ee1c9ada0e3a29d20c78054798f902f44f0e326e


Next, we need to spin our NSO container using this network.

We will spin it based on the ```system``` image that we created before.

In [12]:
!docker run -itd --network nsotestenvnetwork --env-file pipeline-utils/nso_setup.list --platform=linux/amd64 -p 8080:80 --name STAGING-NSO system/nso:6.0-root-alfsando

3d38c0bf096a147b1118b247341f3cda1f5f5e59697b37e0faaffbbe95919090


After it warmed up, we load our xml which contains the definition of the authgroup required for onboarding the netsims:

In [15]:
!docker cp pipeline-utils/netsim_authgroup.xml STAGING-NSO:/
!docker exec -i STAGING-NSO bash -l -c "ncs_load -l -m netsim_authgroup.xml"

Let's spin two more netsims, this time using the network that we created before:

In [20]:
!docker run -td --network-alias nsotestenvnetwork --hostname IOSXR-1 --platform=linux/amd64 -p 223:22 --network nsotestenvnetwork --name IOSXR-1 ned/netsim-cisco-iosxr-cli-7.43:6.0-root-alfsando
!docker run -td --network-alias nsotestenvnetwork --hostname IOSXR-2 --platform=linux/amd64 -p 224:22 --network nsotestenvnetwork --name IOSXR-2 ned/netsim-cisco-iosxr-cli-7.43:6.0-root-alfsando

b9acd8a28903ff2d090f2f4ef2558e10032ea8eb2c88851eca8a38acf9050bf1
0d959e99972258de44fa4b7475d6560bef9f9ad04dbb783a90a412031fa8486a


Once the netsim containers are ready, we can proceed with the onboarding into our staging NSO. 

First of all, let's generate and copy the configuration xml files of each:

In [25]:
!docker exec -i IOSXR-1 bash -l -c "cd /opt/ncs/current/ && ncs-netsim ncs-xml-init > device-1.xml"
!docker exec -i IOSXR-2 bash -l -c "cd /opt/ncs/current/ && ncs-netsim ncs-xml-init > device-2.xml"

Next, we will copy these xml files and update them with the IP of each device within the Docker network, so that NSO is able to reach them without any issues:

In [21]:
!docker cp IOSXR-1:/opt/ncs/current/device-1.xml .
!docker cp IOSXR-2:/opt/ncs/current/device-2.xml .

In [29]:
%%bash
NETSIM_IP_1=$(docker inspect -f '{{ .NetworkSettings.Networks.nsotestenvnetwork.IPAddress}}' IOSXR-1)
NETSIM_IP_2=$(docker inspect -f '{{ .NetworkSettings.Networks.nsotestenvnetwork.IPAddress}}' IOSXR-2)

xmlstarlet edit --inplace -N x=http://tail-f.com/ns/ncs --update '/x:devices/x:device/x:address' --value $NETSIM_IP_1 device-1.xml
xmlstarlet edit --inplace -N x=http://tail-f.com/ns/ncs --update '/x:devices/x:device/x:address' --value $NETSIM_IP_2 device-2.xml

cat device-1.xml
cat device-2.xml

<?xml version="1.0"?>
<devices xmlns="http://tail-f.com/ns/ncs">
  <device>
    <name>IOSXR-1</name>
    <address>172.23.0.3</address>
    <port>22</port>
    <ssh>
      <host-key>
        <algorithm>ssh-ed25519</algorithm>
        <key-data>ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAICp3BzdAicVmmH9CjKDfAstEEi48y3+rwA3YBh4fXoQS root@buildkitsandbox</key-data>
      </host-key>
      <host-key>
        <algorithm>ssh-rsa</algorithm>
        <key-data>ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDsa73No6ya/JHsyAFiaVPbU/GiZnTSTUaOmKCX75KkFWxG5prV0UWKdDd1r33dI9ImJ/NTXsGujJeg6Ln8qQkO7V7R7u/nbBCv5JMAFS42ssGbcddP+z5T9CADIm8xSC30W1xvoLw2552JQQ3cK+DKMyWVxO81PeN1Hf0Lev4EGgnKn92FZDfPNv8NA8YT5j/pYC1hI3CpiWyrBF13cUSTsz5UwAY9KHPUphVobzjecjB+nA/DZXTyHxAE3oI7nzcCegrZQ2da7XyMp02l82H8zYIbCzSAVlCY8pGJR9S/V2AdTTXFgWleulro+Gwy+ABKCLQepu7MmiPJ0HkX0QJ5 root@buildkitsandbox</key-data>
      </host-key>
    </ssh>
    <ssh-algorithms>
      <public-key>ssh-ed25519</public-key>
      <public-key>ssh-rsa</public-key>
    </s

Finally, these updated xml files are copied into the NSO container and the configs are loaded into the server:

In [30]:
!docker cp device-1.xml STAGING-NSO:/
!docker cp device-2.xml STAGING-NSO:/

In [31]:
!docker exec -i STAGING-NSO bash -l -c "ncs_load -l -m device-1.xml"
!docker exec -i STAGING-NSO bash -l -c "ncs_load -l -m device-2.xml"

Last but not least, we sync the devices and show the device list:

In [33]:
!docker exec -i STAGING-NSO bash -l -c "echo 'devices device * sync-from' | ncs_cli -Cu admin"
!docker exec -i STAGING-NSO bash -l -c "echo 'show devices list' | ncs_cli -Cu admin"

devices device IOSXR-1 sync-from
    result true
devices device IOSXR-2 sync-from
    result true
NAME     ADDRESS     DESCRIPTION  NED ID                ADMIN STATE  
-------------------------------------------------------------------
IOSXR-1  172.23.0.3  -            cisco-iosxr-cli-7.43  unlocked     
IOSXR-2  172.23.0.4  -            cisco-iosxr-cli-7.43  unlocked     


### 🙌🏽 And that's it! 🙌🏽

We can proceed to use our fully geared NSO container for development or testing purposes.

Needless to say, to perform a cleanup we just need to destroy the corresponding containers with the following command:

```
docker rm -f <NAME OF THE CONTAINER>

```